In [ ]:
import numpy as np
with open('input.txt') as file:
    lines = file.readlines()

In [ ]:
# generate height map with integers
S_position = 0
E_position = 0
height_map = np.zeros((len(lines), len(lines[0])-1)).astype('int')
for i_line, line in enumerate(lines):
    for i_char, char in enumerate(line):
        if char == '\n':
            continue
        if char == 'S':
            S_position = np.array([i_line, i_char])
            char = 'a'
        if char == 'E':
            E_position = np.array([i_line, i_char])
            char = 'z'
        height_map[i_line, i_char] = ord(char) - ord('a')
print(height_map)
print(S_position)
print(E_position)
print(height_map.shape)

# Watershed-like approach

In [ ]:
# find all positions with elevation 0
# important for part 2 of this day
part = 2
if part == 1:
    start_positions = np.transpose([S_position]) #[np.array(v) for v in S_position]
if part == 2:
    start_positions = np.where(height_map == 0)

In [ ]:
def get_4_neighboorhood_from_tuple_coords(tuple_coords):
    position_offsets = np.array([[i, j] for i in range(-1,2) for j in range(-1,2) if abs(i) != abs(j)])
    transposed = np.transpose(tuple_coords)
    n_coords = len(transposed)
    coords_array = np.repeat(transposed, 4, axis=0)
    offset_array = np.tile(position_offsets, (n_coords,1))
    new_positions = coords_array + offset_array
    reshaped = np.reshape(new_positions, [n_coords, 4, 2])
    return reshaped

In [ ]:
# watershed approach
watershed = np.zeros_like(height_map)
watershed[tuple(start_positions)] = 1
top_reached = False
last_positions = start_positions
iterations = 0
while not top_reached:
    next_positions = get_4_neighboorhood_from_tuple_coords(last_positions)
    valid_positions = np.array([[all(height_map.shape > next_position) and all(next_position >= 0) # next positions exist in height map?
                    and height_map[tuple(next_position)] <= height_map[tuple(last_position)]+1 # next positions are not too high
                    and not watershed[tuple(next_position)] # next positions were not visited before
                    for next_position in four_neighborhood]
                    for last_position, four_neighborhood in zip(np.transpose(last_positions), next_positions)])
    last_positions = np.unique(np.transpose(next_positions[valid_positions]), axis=1)
    watershed[tuple(last_positions)] = iterations + 2
    iterations += 1
    top_reached = any([all(position == E_position) for position in np.transpose(last_positions)])

In [ ]:
print(watershed)
print('shortest trail: ' + str(iterations))